In [5]:
from astroquery.skyview import SkyView
import matplotlib.pyplot as plt
from astropy.table import Table
import numpy as np
import photutils
import scipy.ndimage as ndi
import statmorph


control = Table.read('./stat/control_stat_805.fits')

In [8]:
def morph(image):
    threshold = photutils.detect_threshold(image, 3)
    npixels = 10  # minimum number of connected pixels
    segm = photutils.detect_sources(image, threshold, npixels)
    # Keep only the largest segment
    label = np.argmax(segm.areas) + 1
    segmap = segm.data == label
    segmap_float = ndi.uniform_filter(np.float64(segmap), size=10)
    segmap = segmap_float > 0.5
    source_morphs = statmorph.source_morphology(
        image, segmap,gain=100.0)
    morph = source_morphs[0]
    concen = morph.concentration
    return concen

In [14]:
con_list = []
for num in range(np.shape(control['num'])[0]):
    image_list = SkyView.get_images(position=str(control['objra'][num])+','+str(control['objdec'][num]),
                                    survey='SDSSr',
                                    pixels='200')
    image = image_list[0][0].data
    
    con_list.append(morph(image))

KeyboardInterrupt: 

In [15]:
con_list

[2.4022675855373263,
 3.370425080720904,
 2.7520433203974966,
 3.101133291377289,
 4.032998331808927]